Pyspark practise project Data analysis

In [1]:
# Installing required packages

!pip install wget pyspark  findspark

In [2]:
import findspark

findspark.init()

In [3]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the SparkContext.   
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [4]:
import wget

In [5]:
# Creating a SparkContext object

sc = SparkContext.getOrCreate()

# Creating a Spark Session

spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/09/23 13:34:41 WARN Utils: Your hostname, gabriel-BOD-WXX9 resolves to a loopback address: 127.0.1.1; using 192.168.204.211 instead (on interface wlp0s20f3)
24/09/23 13:34:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/23 13:34:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
url1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv" 
url2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv" 

# Getting data from external source

In [7]:
dataset1 = wget.download(url1)
dataset2 = wget.download(url2)

In [8]:
df1=spark.read.option("header","true").csv("./data/dataset1.csv")
df1.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- date_column: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)



In [9]:
df2 = spark.read.option("header","true").csv("./data/dataset2.csv")
df2.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- value: string (nullable = true)
 |-- notes: string (nullable = true)



In [10]:
from pyspark.sql.functions import year, quarter, to_date,col

# casting to date format


In [12]:
df1.show(3)

df1 =df1.withColumn("date_column",to_date(col("date_column"), "d/M/yyyy"))
df2 =df2.withColumn("transaction_date",to_date(col("transaction_date"), "d/M/yyyy"))

+-----------+-----------+------+-----------+--------+
|customer_id|date_column|amount|description|location|
+-----------+-----------+------+-----------+--------+
|          1| 2022-01-01|  5000| Purchase A| Store A|
|          2| 2022-02-15|  1200| Purchase B| Store B|
|          3| 2022-03-20|   800| Purchase C| Store C|
+-----------+-----------+------+-----------+--------+
only showing top 3 rows



**adding year and quarter column to respectively df1 and df2**

In [15]:
df1=df1.withColumn("year",year(col("date_column")))
df2=df2.withColumn("quarter",quarter(col("transaction_date")))

**renaming columns**

In [23]:
df1=df1.withColumnRenamed("amount","transaction_amount")
df2 =df2.withColumnRenamed("value","transaction_value")

In [24]:
df1.show(3)
df2.show(3)

+-----------+-----------+------------------+-----------+--------+----+
|customer_id|date_column|transaction_amount|description|location|year|
+-----------+-----------+------------------+-----------+--------+----+
|          1| 2022-01-01|              5000| Purchase A| Store A|2022|
|          2| 2022-02-15|              1200| Purchase B| Store B|2022|
|          3| 2022-03-20|               800| Purchase C| Store C|2022|
+-----------+-----------+------------------+-----------+--------+----+
only showing top 3 rows

+-----------+----------------+-----------------+------+-------+
|customer_id|transaction_date|transaction_value| notes|quarter|
+-----------+----------------+-----------------+------+-------+
|          1|      2022-01-01|             1500|Note 1|      1|
|          2|      2022-02-15|             2000|Note 2|      1|
|          3|      2022-03-20|             1000|Note 3|      1|
+-----------+----------------+-----------------+------+-------+
only showing top 3 rows



**droping columns**

In [27]:
df1=df1.drop("description","location")
df2 =df2.drop("notes")

In [42]:
joined_df = df1.join(df2, 'customer_id','inner')
joined_df.show(3)

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|year|transaction_date|transaction_value|quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1| 2022-01-01|              5000|2022|      2022-01-01|             1500|      1|
|          2| 2022-02-15|              1200|2022|      2022-02-15|             2000|      1|
|          3| 2022-03-20|               800|2022|      2022-03-20|             1000|      1|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
only showing top 3 rows



**filtered df**

In [49]:
filtered_df = joined_df.filter( joined_df["transaction_amount"] >1000)
filtered_df.show(3)

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|year|transaction_date|transaction_value|quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1| 2022-01-01|              5000|2022|      2022-01-01|             1500|      1|
|          2| 2022-02-15|              1200|2022|      2022-02-15|             2000|      1|
|          4| 2022-04-10|              3000|2022|      2022-04-10|             2500|      2|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
only showing top 3 rows



**aggregate data by customer**

In [47]:
from pyspark.sql.functions import sum

In [56]:
total_ammount_per_customer= filtered_df.groupBy('customer_id').agg(sum('transaction_amount'))
total_ammount_per_customer.show(5)

+-----------+-----------------------+
|customer_id|sum(transaction_amount)|
+-----------+-----------------------+
|         51|                 4200.0|
|         15|                 4200.0|
|         54|                 1500.0|
|         11|                 2200.0|
|         69|                 5500.0|
+-----------+-----------------------+
only showing top 5 rows



**writte table to H table**

In [57]:
total_ammount_per_customer.write.mode("overwrite").saveAsTable("customer_totals")

In [58]:
total_ammount_per_customer.write.parquet("total_ammount_per_customer.parquet")

**add new column**

In [61]:
df1.show(3)

+-----------+-----------+------------------+----+
|customer_id|date_column|transaction_amount|year|
+-----------+-----------+------------------+----+
|          1| 2022-01-01|              5000|2022|
|          2| 2022-02-15|              1200|2022|
|          3| 2022-03-20|               800|2022|
+-----------+-----------+------------------+----+
only showing top 3 rows



In [60]:
from pyspark.sql.functions import when,lit

**add column**

In [65]:
df1 = df1.withColumn("high_value", when(col("transaction_amount")  >= 5000,lit("Yes")).otherwise(lit("No")))
df1.show(3)

+-----------+-----------+------------------+----+----------+
|customer_id|date_column|transaction_amount|year|high_value|
+-----------+-----------+------------------+----+----------+
|          1| 2022-01-01|              5000|2022|       Yes|
|          2| 2022-02-15|              1200|2022|        No|
|          3| 2022-03-20|               800|2022|        No|
+-----------+-----------+------------------+----+----------+
only showing top 3 rows



**calculate average by qurter**

In [ ]:
df2.show(3)

+-----------+----------------+-----------------+-------+
|customer_id|transaction_date|transaction_value|quarter|
+-----------+----------------+-----------------+-------+
|          1|      2022-01-01|             1500|      1|
|          2|      2022-02-15|             2000|      1|
|          3|      2022-03-20|             1000|      1|
+-----------+----------------+-----------------+-------+
only showing top 3 rows



In [71]:
from pyspark.sql.functions import avg

average_value_per_quarter=df2.groupBy('quarter').agg(avg("transaction_value").alias("average_trans_val"))

In [72]:
average_value_per_quarter.write.mode("overwrite").saveAsTable("quarterly_averages")

**total transaction value per year**

In [73]:
df1.show(3)

+-----------+-----------+------------------+----+----------+
|customer_id|date_column|transaction_amount|year|high_value|
+-----------+-----------+------------------+----+----------+
|          1| 2022-01-01|              5000|2022|       Yes|
|          2| 2022-02-15|              1200|2022|        No|
|          3| 2022-03-20|               800|2022|        No|
+-----------+-----------+------------------+----+----------+
only showing top 3 rows



In [78]:
total_value_per_year = df1.groupBy("year").agg(sum("transaction_amount").alias("total_transaction_val"))

In [79]:
total_value_per_year.write.mode("overwrite").csv("total_value_per_year.csv")